###### A company produces a set of products at I plants. It then ships these products to J market zones. For i = 1,...,I y j = 1,...,J, the following data are given:
**vi** ≡ variable cost of producing one unit of product at plant i. 
**cij** ≡ cost of shipping one unit of product from plant i to market j. 
**dj** ≡ demand for products at market j. 
**Mi** ≡ maximum number or products produced at plant i. 
**pj** ≡ selling price for products at market j. 
**F** ≡maximum transportation capacity between plant i and market j (assume all transportation capacities are equal).


**a)** (3 points) Formulate the problem (linear program) of maximizing the total proﬁt that the company is facing (total selling revenue - total production and transportation costs) by identifying the optimal production and transportation schedule. Assume also that demand is satisfy with “equality”, i.e., “total product reaching market j” = dj).

$$max  \sum_{i=1}^{I}\sum_{j=1}^{J} Xij*Pj - Xij*Vi - Xij*Cij  $$

$\displaystyle \sum_{i=1}^{I} X_{ij} = d_{j} \quad j=1,...,J \quad $  

$\displaystyle \sum_{j=1}^{J} X_{ij} \le M_{i} \quad i=1,...,I \quad $  

$X_{ij} \le F \quad j=1,...,J \quad i=1,...,I \quad $ 

$X_{ij} \ge 0 \quad j=1,...,J \quad i=1,...,I \quad $ 

**b)** (4 points) Implement the model in Pyomo as an “AbstractModel()” and solve it considering 5 plants and 6 markets and randomly generated parameters. For the random generation of these parameters consider uniform distributions within the following ranges: vi ∈ [1.5,2.5], ci,j ∈ [1,2], dj ∈ [275,325], Mi ∈ [400,900], pj ∈ [4,5] y F ∈ [250,350]. 

In [53]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

################ DATA #####################

i=5; #number of factories
j=6; #number of markets

#variable cost of producing one unit of product at plant i.
v_i = np.random.uniform(1.5, 2.5, size=i)

#cost of shipping one unit of product from plant i to market j.
c_ij = np.random.random_integers(1, 2, size=(i,j))

#demand for products at market j.
d_j = np.random.random_integers(275, 325, size=j)

#maximum number or products produced at plant i.
m_i = np.random.random_integers(400, 900, size=i)

#selling price for products at market j.
p_j = np.random.random_integers(4, 5, size=j)

#maximum transportation capacity between plant i and market j.
F = np.random.random_integers(250, 350)

C:\Users\alper\Anaconda2\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. Please call randint(1, 2 + 1) instead
  
C:\Users\alper\Anaconda2\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: This function is deprecated. Please call randint(275, 325 + 1) instead
C:\Users\alper\Anaconda2\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: This function is deprecated. Please call randint(400, 900 + 1) instead
C:\Users\alper\Anaconda2\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: This function is deprecated. Please call randint(4, 5 + 1) instead
C:\Users\alper\Anaconda2\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: This function is deprecated. Please call randint(250, 350 + 1) instead


In [54]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
opt = SolverFactory("glpk")

def trans_opt(n,m,v_i,c_ij,d_j,m_i,p_j,F):

   ###########################################

 #return instance.OBJ(), x_sol

    model = AbstractModel()

    model.I = RangeSet(0,n-1)
    model.J = RangeSet(0,m-1)

    # the next line declares a variable indexed by the set J 
    model.x = Var(model.I,model.J, domain=NonNegativeReals)

    #definition of the objective function
    def obj_expression(model): 
        return sum(sum(p_j[j]*model.x[i,j] for i in model.I) for j in model.J)-sum(
            sum(v_i[i]*model.x[i,j] for j in model.J) for i in model.I)-sum(
            sum(c_ij[i,j]*model.x[i,j] for j in model.J) for i in model.I)

    model.OBJ = Objective(rule=obj_expression, sense=maximize)

    #maximum transportation capacity
    def flow_constraint_rule(model,i,j):
        return model.x[i,j] <= F
    model.max_flow = Constraint(model.I, model.J, rule=flow_constraint_rule)


    #minimum demand constraint
    def b_constraint_rule(model, j): # return the expression for the constraint for i
        return sum(model.x[i,j] for i in model.I)== d_j[j]

    # the next line creates one constraint for each member of the set model.J 
    model.dem_Constraint = Constraint(model.J, rule=b_constraint_rule)

    #maximum production constraint
    def a_constraint_rule(model, i): # return the expression for the constraint for i
        return sum(model.x[i,j] for j in model.J)<= m_i[i]

    model.prod_Constraint = Constraint(model.I, rule=a_constraint_rule)

    instance = model.create_instance()
    results=opt.solve(instance)


    x_sol = np.zeros((n,m))
    for a in range(0,n):
        for b in range(0,m):
            x_sol[a,b] = instance.x[a,b].value


    return(instance.OBJ(),x_sol)

In [59]:
[obj,x_sol]=trans_opt(i,j,v_i,c_ij,d_j,m_i,p_j,F)
print(obj)
print(x_sol)

3153.54010061
[[   0.   35.    0.   44.  250.  250.]
 [   0.  250.    0.    0.   70.    0.]
 [ 213.    0.    0.  250.    0.    0.]
 [  95.    0.   74.    0.    0.    0.]
 [   0.    0.  250.    0.    0.   57.]]


**c)** Considering the results: which plant’s maximum capacity would be more beneﬁcial to increase? and in which market would be more beneﬁcial to increase the demand? Justify your answers.

In [49]:
print("Variables: \n\n v_i: {} \n\n c_ij: {} \n\n d_j: {} \n\n m_i: {} \n\n p_i: {} \n\n F: {}".format(v_i,c_ij,d_j,m_i,p_j,F))

Variables: 

 v_i: [ 1.59701303  1.93347299  1.56222914  2.20606982  2.34322107] 

 c_ij: [[1 2 2 2 1 1]
 [2 2 2 2 2 1]
 [2 2 2 1 2 2]
 [2 1 2 1 1 1]
 [1 1 1 2 1 1]] 

 d_j: [314 280 322 296 314 313] 

 m_i: [454 476 618 568 777] 

 p_i: [4 5 5 5 4 5] 

 F: 312


For the values above, the first plant provides the maximum profit that you can obtain from the objective matrix. Summing the weights of all plants on matrix shows that the first plant has the most benefit.
[   0.   35.    0.   44.  250.  250.]

The formula to obtain the most beneficial market is the same with the most benficial plant. We can sum the values for each market, and see the most beneficial market to increase the demand. The second market is the most beneficial one to increase demand.
[ 250.
  70.
  0.
  0.
  0.]


**d)** Modify the formulation in a) to impose that:

    1. Each plant is allowed to send its products to a maximum of 5 diferent markets.
    2. Plants, if operative, cannot produce less than 10% of their maximum capacity, i.e., a plant can produce either 0 products (if not operative) or a quantity greater than 10% of its maximum capacity (if operative).

Solve the resulting model in Pyomo and interpret the new results. 

$X_{ij} \equiv$ units produced in market $i$ and sold in market $j$.  

$A_{ij} \equiv$ if the plant $i$ sends units to market $j$ 1, if not 0.

$B_{i} \equiv$ if the plant $i$ is active 1, if not 0. 



 $$max  \sum_{i=1}^{I}\sum_{j=1}^{J} Xij*Pj - Xij*Vi - Xij*Cij  $$



$\displaystyle \sum_{i=1}^{I} X_{ij} = d_{j} \quad j=1,...,J \quad $  

$\displaystyle \sum_{j=1}^{J} X_{ij} \le M_{i} B_{i} \quad i=1,...,I \quad $  

$\displaystyle \sum_{j=1}^{J} X_{ij} \ge 0.1 M_{i} B_{i}\quad i=1,...,I \quad $  

$X_{ij} \le A_{ij} F  \quad j=1,...,J \quad i=1,...,I \quad $ 

$X_{ij} \ge 0 \quad j=1,...,J \quad i=1,...,I \quad $

$\displaystyle \sum_{j=1}^{J} A_{ij} \le 5 \quad i=1,...,I \quad $

In [56]:
from pyomo.environ import *
opt = SolverFactory("glpk")
from pyomo.opt import SolverFactory

def trans_opt_second(n,m,v_i,c_ij,d_j,m_i,p_j,F):
    
    model = AbstractModel()
    
    model.I = RangeSet(0,n-1)
    model.J = RangeSet(0,m-1)
    
    #We create I·J non negative variables 
    model.x = Var(model.I,model.J, domain=NonNegativeReals) 
    
    #We create I·J binary variables
    model.a = Var(model.I,model.J, domain=Binary)
    
    #We create I binary variables
    model.b = Var(model.I, domain=Binary)
    
    #definition of the objective function
    def obj_expression(model): 
        return sum(sum(p_j[j]*model.x[i,j] for i in model.I) for j in model.J)-sum(
            sum(v_i[i]*model.x[i,j] for j in model.J) for i in model.I)-sum(
            sum(c_ij[i,j]*model.x[i,j] for j in model.J) for i in model.I)
     
    model.OBJ = Objective(rule=obj_expression, sense=maximize)
    
    #maximum transportation capacity
    def flow_constraint_rule(model,i,j):
        return model.x[i,j] <= F*model.a[i,j]
    model.max_flow = Constraint(model.I, model.J, rule=flow_constraint_rule)

    #minimum demand constraint
    def b_constraint_rule(model, j): # return the expression for the constraint for i
        return sum(model.x[i,j] for i in model.I)== d_j[j]

    # the next line creates one constraint for each member of the set model.J 
    model.dem_Constraint = Constraint(model.J, rule=b_constraint_rule)

    #Upper bound production constraint
    def M_constraint_rule(model, i): 
        return sum(model.x[i,j] for j in model.J)<= m_i[i-1]*model.b[i]
    
    #the next line creates one constraint for each of the I plants
    model.M_constraint = Constraint(model.I, rule=M_constraint_rule)
    
    #Lower bound production constraint
    def Ml_constraint_rule(model, i): 
        return sum(model.x[i,j] for j in model.J)>= 0.1*m_i[i-1]*model.b[i]
    
    #the next line creates one constraint for each of the I plants
    model.Ml_constraint = Constraint(model.I, rule=Ml_constraint_rule)
    
    #No more than 5 markets constraint
    def Y_constraint_rule(model,i): 
        return   sum(model.a[i,j] for j in model.J) <= 5
    
    #the next line creates one constraint for each of the I plants
    model.Y_constraint = Constraint(model.I,rule=Y_constraint_rule)
    

    instance = model.create_instance()
    results = opt.solve(instance)
    

    x_sol = np.zeros((n,m))
        
    for i in range(0, n):
            for j in range(0,m):
                x_sol[i,j]=instance.x[i,j].value
    
    a_sol = np.zeros((n,m))
    
    for i in range(0, n):
            for j in range(0,m):
                a_sol[i,j]=instance.a[i,j].value
                
    b_sol = np.zeros(n)
    
    for i in range(0, n):
        b_sol[i]=instance.b[i].value           
    
    return instance.OBJ(), x_sol, a_sol, b_sol

In [60]:
[obj,x_sol,a_sol,b_sol]=trans_opt_second(i,j,v_i,c_ij,d_j,m_i,p_j,F)
print("Obj: {}".format(obj)+"\n")
print("x_sol:\n {}".format(x_sol)+"\n")
print("a_sol:\n {}".format(a_sol)+"\n")
print("b_sol:\n {}".format(b_sol)+"\n")

Obj: 3222.72059983

x_sol:
 [[   0.   35.    0.   44.  250.  250.]
 [   0.  250.    0.    0.   70.    0.]
 [ 250.    0.    0.  250.    0.   57.]
 [  58.    0.   74.    0.    0.    0.]
 [   0.    0.  250.    0.    0.    0.]]

a_sol:
 [[ 0.  1.  0.  1.  1.  1.]
 [ 1.  1.  1.  0.  1.  0.]
 [ 1.  0.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]]

b_sol:
 [ 1.  1.  1.  1.  1.]



Here we can compare two results for two different models.
For the second model, I obtain greater objective function value.